## Milvus Container 실행
milvus 디렉토리 내부에 필요 파일들 설치

In [ ]:
# cd milvus/
# curl -sfL https://raw.githubusercontent.com/milvus-io/milvus/master/scripts/standalone_embed.sh -o standalone_embed.sh
# bash standalone_embed.sh start
# docker pull zilliz/attu:latest
# docker run -p 8000:3000 -e HOST_URL=http://127.0.0.1:8000 -e MILVUS_URL={Default Swtich의 IPv4 주소}:19530 zilliz/attu:latest
# cd ../

## Milvus Database Create

In [90]:
from pymilvus import connections, utility, FieldSchema, CollectionSchema, DataType, Collection

QUERY_MODEL = 'solar-embedding-1-large-query'
PASSAGE_MODEL = 'solar-embedding-1-large-passage'

DENSE_FIELD = "dense_vector"
DENSE_DIMENSION = 4096 
DENSE_INDEX = {"index_type" : "FLAT", "metric_type" : "IP"}

SPARSE_FILED = "sparse_vector"
SPARSE_INDEX = {"index_type" : "SPARSE_INVERTED_INDEX", "metric_type" : "IP"}

# Collection 접속 및 생성
def milvus_connect(collection_name):
    # Milvus 설정
    MILVUS_HOST = '127.0.0.1'
    MILVUS_PORT = '19530'     

    # Milvus 연결
    connections.connect(host=MILVUS_HOST, port=MILVUS_PORT)
    if connections:
        print("Milvus connected")
    else:
        exit()

    # 컬렉션 존재 여부 확인 및 생성
    if utility.has_collection(collection_name):
        collection = Collection(collection_name)
        print(f"Collection '{collection_name}' loaded.")
    else:
        print(f"Collection '{collection_name}' does not exist.")
        print(f"Create '{collection_name}' Collection.")
        
        field_args = [
            FieldSchema(name='id', dtype=DataType.VARCHAR, is_primary=True, max_length=100),        # split된 block의 고유 ID
            FieldSchema(name='title', dtype=DataType.VARCHAR, max_length=2000),
            FieldSchema(name='date', dtype=DataType.INT64),
            FieldSchema(name='content', dtype=DataType.VARCHAR, max_length=5000),
            FieldSchema(name='NER', dtype=DataType.VARCHAR, max_length=3000),
            FieldSchema(name=DENSE_FIELD, dtype=DataType.FLOAT_VECTOR, dim=DENSE_DIMENSION),
            FieldSchema(name=SPARSE_FILED, dtype=DataType.SPARSE_FLOAT_VECTOR)
        ]
        
        # 스키마 정의
        schema = CollectionSchema(fields=field_args)
        
        # 컬렉션 생성
        collection = Collection(name=collection_name, schema=schema)
        
        # 인덱스 생성
        collection.create_index("dense_vector", DENSE_INDEX)
        collection.create_index("sparse_vector", SPARSE_INDEX)

        collection.flush()

        print(f"Collection '{collection_name}' created and loaded.")
    # 생성된 컬렉션 반환
    return collection

# Collection 제거용
def drop_collection(collection_name):
    MILVUS_HOST = '127.0.0.1'
    MILVUS_PORT = '19530' 
    connections.connect(host=MILVUS_HOST, port=MILVUS_PORT)
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)
        print(f"Drop {collection_name} Complete.")
    else:
        print(f"Can't find {collection_name}.")
# drop_collection("Donga")

In [91]:
drop_collection("Donga")

Can't find Donga.


In [92]:
collection = milvus_connect("Donga")
print(collection)

Milvus connected
Collection 'Donga' does not exist.
Create 'Donga' Collection.
Collection 'Donga' created and loaded.
<Collection>:
-------------
<name>: Donga
<description>: 
<schema>: {'auto_id': False, 'description': '', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 100}, 'is_primary': True, 'auto_id': False}, {'name': 'title', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 2000}}, {'name': 'date', 'description': '', 'type': <DataType.INT64: 5>}, {'name': 'content', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 5000}}, {'name': 'NER', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 3000}}, {'name': 'dense_vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 4096}}, {'name': 'sparse_vector', 'description': '', 'type': <DataType.SPARSE_FLOAT_VECTOR: 104>}], 'enable_dynamic_field': False}



## Data Load & Embedding Function Setting

In [77]:
# Sparse Embedding : Custom BM25

from model.bm25_model import Custom_BM25, Kiwi_Tokenizer

tokenizer = Kiwi_Tokenizer("model/custom_dict.txt")
custom_bm25 = Custom_BM25(corpus=[], tokenizer=tokenizer)
custom_bm25.load("model/bm25_model.json")
print(custom_bm25)

In [78]:
# Dense Embedding : Upstage/solar-embedding-1-large

from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

client = OpenAI(
    api_key = os.getenv("UPSTAGE_API_KEY"),
    base_url = "https://api.upstage.ai/v1/solar"
)

In [79]:
import pandas as pd

df = pd.read_csv("dataset/2024_norm_split_ner_pruning.csv")
print(len(df))
df.head()

72401


,id,title,date,content,NER
0,122846231_0,중 군 고위간부 9명 전인대 대표 박탈… '로켓군' 납품 관련 부패사건 연루 의혹,20240101,"""리상푸 전국방부장 낙마와 연관"" 리부장 후임엔 해군출신 둥쥔 중국 당국이 '반(반...",리상푸 전국방부장 리부장 해군 둥쥔 중국 군 중국 국방부장 대만 필리핀 남중국해 중...
1,122846231_1,중 군 고위간부 9명 전인대 대표 박탈… '로켓군' 납품 관련 부패사건 연루 의혹,20240101,장비 발전(조달) 부문에 집중됐던 것으로 전해졌다. 커우젠원(구건문) 대만정치대 ...,커우젠원 구건문 대만정치대 주임 전국인대 중국 중앙군사위 중국 리상푸 李상복 국방부...
2,122846362_0,"서울 12cm 기습 폭설… ""이런 눈 또 온다""",20240101,"13년만에 최대… 도로 곳곳 사고 방파제 추락-조난사고도 발생 ""엘니뇨-온난화로 폭...",서울 기상청 서울 한반도 강원 서울 광진구 마을버스 종로구 서대문구
3,122846362_1,"서울 12cm 기습 폭설… ""이런 눈 또 온다""",20240101,역 방향으로 향하는 통일로 5차로에서 추돌사고가 발생해 도로가 한때 부분 통제됐다....,통일로 5차로 천호대교 북단 강원 춘천시 트럭 강원 삼척시 아래 구급대원 평창군 소...
4,122846400_0,"'자산 301조원' 머스크, 세계 최고 부자에",20240101,1년새 124조원 늘어… 증가액도 1위 아르노 LVMH 회장 2위로 밀려나 베이조스...,아르노 LVMH 회장 베이조스 게이츠 머어이재용 회장 일론 프랑스 루이뷔통모에에네시...


In [93]:
import gc
from tqdm import tqdm

start = 0
end = 100
iteration = len(df) // 100 + 1
for offset in tqdm(range(iteration)): # upstage에 100개씩만 허용이 되어서
    # 데이터 슬라이싱
    rows = df.iloc[start + 100 * offset : end + 100 * offset]
    contents = rows['content'].tolist()

    # Dense Embedding 생성 (list)
    dense_embedding = client.embeddings.create(
        model = 'solar-embedding-1-large-passage',
        input = contents
    )
    dense_vectors = [doc_embedding.embedding for doc_embedding in dense_embedding.data]
    
    # Sparse Embedding 생성 (dict)
    sparse_embedding = custom_bm25.embed_documents(contents)

    # Entities 생성 및 Milvus 삽입
    entities = []
    for idx in range(len(rows)):
        entity = {
            "id" : rows.iloc[idx]['id'],
            "title" : rows.iloc[idx]['title'],
            "date" : rows.iloc[idx]['date'],
            "content" : rows.iloc[idx]['content'],
            "NER" : rows.iloc[idx]['NER'],
            DENSE_FIELD : dense_vectors[idx],
            SPARSE_FILED : sparse_embedding[idx],
        }
        entities.append(entity)
    collection.insert(entities) # 100개씩 넣기
    collection.load()

    # 사용이 끝난 변수 해제
    del dense_vectors
    del sparse_embedding
    del entities
    gc.collect()

# 최종 메모리 해제
gc.collect()

  0%|          | 0/725 [00:00<?, ?it/s]

100%|██████████| 725/725 [4:12:32<00:00, 20.90s/it]  


0